<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9c72e46d458e3c3138feb8e8f1f5e2ffc0f3d0c1bb1c78ec3bb27bf85d7e002a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-26 09:58:10
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.42 C
-------------------
Today PnL: -1.56 K (-0.01%)
Current PnL: -25.42 L (-16.42%)
CY Booked + Current PnL: -11.13 L (-7.19%)
-------------------
Total profit:  1.59 L
Total loss:  -27.02 L
-------------------
Total Booked + Current PnL: 15.85 L (12.4%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.09%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.06 L (63.59%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.63%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.50,-18.83,23.32,0.09,19075.0,-18981.0,81796.0,89.75,41.0,M-SC,2.85,253.0,-1.00,0.58,7.46,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.45,9.60,9.21,19.69,19355.0,18408.0,210150.0,-1.08,58.0,X-MC,2.46,78.0,0.95,1.49,17.05,XY25,NTT,AC
50,MASFIN,398.61,-0.20,-1.87,24.37,22.05,23432.0,-1830.0,96150.0,-15.54,54.0,H-SC,6.89,164.0,-0.08,0.68,39.76,XR,ATH,FINANCE
43,ITC,452.00,-0.37,-0.71,11.58,10.78,27630.0,-1714.0,238604.0,-40.74,53.0,X-LC,2.01,5.0,-0.06,1.69,4.49,X40,NTT,FMCG
52,NESTLEIND,1377.00,-0.55,10.45,9.91,21.40,29052.0,27729.0,293155.0,2.52,53.0,X-LC,5.58,12.0,0.95,2.08,17.78,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TITAGARH,1548.00,4.77,-15.70,72.88,45.74,148785.0,-38026.0,204151.0,12.33,69.0,H-SC,6.19,174.0,-0.26,1.45,32.78,XY24,NTT,ENGINEERING
44,JCHAC,2282.00,2.36,-35.76,55.66,-0.01,45695.0,-45709.0,82096.0,16704.35,38.0,M-SC,12.09,233.0,-1.00,0.58,5.07,OX40N,NTT,AC
66,SIS,528.00,2.22,-21.94,55.34,21.26,47965.0,-24358.0,86674.0,2047.67,61.0,H-SC,4.85,166.0,-0.51,0.61,17.11,OX40N,NTT,MISC
12,BERGEPAINT,680.00,2.04,-1.10,22.49,21.14,50566.0,-2503.0,224836.0,-8.12,55.0,X-MC,1.06,74.0,-0.05,1.60,26.73,XY24,NTT,PAINTS
26,GILLETTE,11206.78,1.91,-0.19,32.67,32.42,82789.0,-486.0,253410.0,-17.85,55.0,X-SC,10.89,85.0,-0.01,1.80,14.88,X40,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
47,KANSAINER,340.0,-2.98,-21.57,44.68,13.47,94498.0,-58167.0,211500.0,-67.73,55.0,H-SC,6.84,158.0,-0.62,1.50,7.54,XY24,NTT,PAINTS
6,ASIANTILES,137.0,-2.27,0.93,80.00,81.67,75745.0,871.0,94681.0,8527.78,71.0,L-SC,14.13,271.0,0.01,0.67,83.35,XR,NTT,CERAMICS
37,IEX,219.0,-1.79,-7.26,60.26,48.63,111743.0,-14520.0,185434.0,-38.37,39.0,H-SC,13.41,136.0,-0.13,1.32,3.45,XR,NTT,MISC
80,VALIANTORG,838.0,-1.30,-42.55,203.13,74.15,190928.0,-69612.0,93993.0,-269.06,53.0,H-SC,21.79,149.0,-0.36,0.67,20.48,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-2.27,0.93,80.00,81.67,75745.0,871.0,94681.0,8527.78,71.0,L-SC,14.13,271.0,0.01,0.67,83.35,XR,NTT,CERAMICS
17,COALINDIA,484.83,0.90,-0.17,19.43,19.23,29184.0,-258.0,150202.0,16.96,77.0,L-LC,2.83,179.0,-0.01,1.07,21.07,XY25,ATH,MINING


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.07,-1.97,112.10,107.91,162241.0,-2915.0,144729.0,-20.15,54.0,M-SC,11.71,216.0,-0.02,1.03,7.91,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.68,-18.59,22.86,0.02,32485.0,-32455.0,142104.0,116.52,36.0,M-SC,10.61,234.0,-1.00,1.01,23.61,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.50,-18.83,23.32,0.09,19075.0,-18981.0,81796.0,89.75,41.0,M-SC,2.85,253.0,-1.00,0.58,7.46,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.98,-21.57,44.68,13.47,94498.0,-58167.0,211500.0,-67.73,55.0,H-SC,6.84,158.0,-0.62,1.50,7.54,XY24,NTT,PAINTS
66,SIS,528.00,2.22,-21.94,55.34,21.26,47965.0,-24358.0,86674.0,2047.67,61.0,H-SC,4.85,166.0,-0.51,0.61,17.11,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-2.27,0.93,80.00,81.67,75745.0,871.0,94681.0,8527.78,71.0,L-SC,14.13,271.0,0.01,0.67,83.35,XR,NTT,CERAMICS
84,WIPRO,420.0,0.01,9.95,56.67,72.27,105585.0,16868.0,186316.0,-5.63,67.0,M-LC,4.17,99.0,0.16,1.32,16.74,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.01,9.95,56.67,72.27,105585.0,16868.0,186316.0,-5.63,67.0,M-LC,4.17,99.0,0.16,1.32,16.74,XR,NTT,IT
6,ASIANTILES,137.00,-2.27,0.93,80.00,81.67,75745.0,871.0,94681.0,8527.78,71.0,L-SC,14.13,271.0,0.01,0.67,83.35,XR,NTT,CERAMICS
25,FINCABLES,1641.55,0.07,-1.97,112.10,107.91,162241.0,-2915.0,144729.0,-20.15,54.0,M-SC,11.71,216.0,-0.02,1.03,7.91,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.20,-3.95,115.22,106.72,136497.0,-4870.0,118466.0,-52.54,41.0,H-SC,4.74,139.0,-0.04,0.84,18.51,AR,ATH,MISC
50,MASFIN,398.61,-0.20,-1.87,24.37,22.05,23432.0,-1830.0,96150.0,-15.54,54.0,H-SC,6.89,164.0,-0.08,0.68,39.76,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-0.27,-25.21,105.25,53.50,237513.0,-76085.0,225666.0,-63.76,30.0,X-MC,6.41,58.0,-0.32,1.60,0.23,XY24,NTT,FMCG
9,BAJAJHFL,181.50,0.04,-18.45,90.65,55.47,179847.0,-44889.0,198397.0,-27.82,30.0,X-MC,16.06,64.0,-0.25,1.41,1.66,X40N,ATH,FINANCE
22,DIXON,18931.72,-1.19,-16.90,51.71,26.07,83888.0,-32994.0,162227.0,-59.57,30.0,X-MC,13.16,56.0,-0.39,1.15,1.04,X40N,ATH,IT
3,ACC,3906.00,-0.31,-27.13,125.26,64.15,217201.0,-64551.0,173400.0,-57.19,32.0,X-SC,4.53,82.0,-0.30,1.23,0.00,XY24,BTT,CEMENT
34,HONAUT,58357.33,0.23,-18.21,75.22,43.32,100208.0,-29652.0,133220.0,-29.66,32.0,X-SC,6.26,90.0,-0.30,0.95,2.29,X40N,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,2.04,-1.10,22.49,21.14,50566.0,-2503.0,224836.0,-8.12,55.0,X-MC,1.06,74.0,-0.05,1.60,26.73,XY24,NTT,PAINTS
15,CAMS,950.0,-0.39,-0.80,25.48,24.48,63660.0,-2013.0,249843.0,-81.09,47.0,X-SC,1.62,86.0,-0.03,1.77,22.15,X40N,NTT,MISC
78,UNITDSPR,1644.0,0.27,4.75,15.29,20.77,37502.0,11128.0,245272.0,-4.44,50.0,X-MC,1.87,62.0,0.30,1.74,12.00,X40N,NTT,BREWERIES
43,ITC,452.0,-0.37,-0.71,11.58,10.78,27630.0,-1714.0,238604.0,-40.74,53.0,X-LC,2.01,5.0,-0.06,1.69,4.49,X40,NTT,FMCG
20,DABUR,735.0,0.00,-4.09,49.83,43.70,118554.0,-10151.0,237917.0,-11.99,33.0,X-MC,2.11,72.0,-0.09,1.69,10.34,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.31,-27.13,125.26,64.15,217201.0,-64551.0,173400.0,-57.19,32.0,X-SC,4.53,82.0,-0.30,1.23,0.00,XY24,BTT,CEMENT
8,AWL,485.00,-0.27,-25.21,105.25,53.50,237513.0,-76085.0,225666.0,-63.76,30.0,X-MC,6.41,58.0,-0.32,1.60,0.23,XY24,NTT,FMCG
19,COLPAL,3726.84,0.01,-20.51,78.02,41.51,163335.0,-54015.0,209350.0,-5.42,37.0,X-MC,8.02,61.0,-0.33,1.49,0.29,XY25,ATH,FMCG
11,BATAINDIA,2096.00,-0.17,-38.85,121.10,35.20,95286.0,-49986.0,78684.0,3.10,35.0,X-SC,17.16,92.0,-0.52,0.56,0.93,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.19,-16.90,51.71,26.07,83888.0,-32994.0,162227.0,-59.57,30.0,X-MC,13.16,56.0,-0.39,1.15,1.04,X40N,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.31,-27.00,95.67,42.84,45400.0,-17551.0,47455.0,-52.07,55.0,X-SC,33.29,83.0,-0.39,0.34,7.36,XY24,NTT,MISC
59,RELAXO,1176.00,-0.39,-47.71,191.27,52.31,145183.0,-69255.0,75905.0,-44.34,43.0,X-SC,7.20,91.0,-0.48,0.54,1.60,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.17,-38.85,121.10,35.20,95286.0,-49986.0,78684.0,3.10,35.0,X-SC,17.16,92.0,-0.52,0.56,0.93,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.25,-0.29,24.43,24.08,30925.0,-365.0,126585.0,-8.38,50.0,X-SC,6.61,89.0,-0.01,0.90,19.27,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.23,-18.21,75.22,43.32,100208.0,-29652.0,133220.0,-29.66,32.0,X-SC,6.26,90.0,-0.30,0.95,2.29,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-0.54,-9.23,32.98,20.71,113229.0,-34912.0,343325.0,-21.68,66.0,X-LC,3.01,1.0,-0.31,2.44,14.72,X40,ATH,IT
41,INFY,1972.00,-0.00,13.24,18.55,34.24,65723.0,41416.0,354304.0,-11.70,67.0,X-LC,6.48,2.0,0.63,2.51,22.62,X40,NTT,IT
75,TMPV,600.00,-0.35,-26.63,67.62,22.98,112067.0,-60167.0,165731.0,-24.30,48.0,X-LC,6.20,3.0,-0.54,1.18,4.24,XY24,NTT,AUTO
81,VBL,671.64,0.36,-3.17,40.11,35.67,122666.0,-10017.0,305825.0,-14.67,55.0,X-LC,3.32,4.0,-0.08,2.17,10.24,X40N,ATH,FMCG
43,ITC,452.00,-0.37,-0.71,11.58,10.78,27630.0,-1714.0,238604.0,-40.74,53.0,X-LC,2.01,5.0,-0.06,1.69,4.49,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.99,-36.77,110.17,32.89,53783.0,-28388.0,48818.0,-702.61,57.0,L-MC,6.92,259.0,-0.53,0.35,34.44,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-2.27,0.93,80.00,81.67,75745.0,871.0,94681.0,8527.78,71.0,L-SC,14.13,271.0,0.01,0.67,83.35,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.20,-1.87,24.37,22.05,23432.0,-1830.0,96150.0,-15.54,54.0,H-SC,6.89,164.0,-0.08,0.68,39.76,XR,ATH,FINANCE
13,BSOFT,831.70,0.88,-17.81,83.46,50.79,96483.0,-25049.0,115604.0,5.53,70.0,H-SC,9.31,151.0,-0.26,0.82,34.60,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.88,-17.81,83.46,50.79,96483.0,-25049.0,115604.0,5.53,70.0,H-SC,9.31,151.0,-0.26,0.82,34.60,XR,ATH,IT
6,ASIANTILES,137.00,-2.27,0.93,80.00,81.67,75745.0,871.0,94681.0,8527.78,71.0,L-SC,14.13,271.0,0.01,0.67,83.35,XR,NTT,CERAMICS
56,RAJESHEXPO,518.00,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
80,VALIANTORG,838.00,-1.30,-42.55,203.13,74.15,190928.0,-69612.0,93993.0,-269.06,53.0,H-SC,21.79,149.0,-0.36,0.67,20.48,XR,NTT,CHEMICALS
17,COALINDIA,484.83,0.90,-0.17,19.43,19.23,29184.0,-258.0,150202.0,16.96,77.0,L-LC,2.83,179.0,-0.01,1.07,21.07,XY25,ATH,MINING


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.36
1,25,44.84
2,50,76.54


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.08
MC    30.74
LC    24.22
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.85
X40      23.07
X40N     12.88
XR        9.88
XY25      9.15
AR        8.87
OX40N     7.59
SR        0.99
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.18
H-SC    23.91
X-LC    20.72
M-SC    11.53
X-SC     8.09
H-MC     4.81
L-SC     1.55
M-MC     1.40
M-LC     1.32
H-LC     1.11
L-LC     1.07
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.23,-7.47,42.02
IT,13.07,-14.11,71.49
FINANCE,9.63,-17.26,68.26
MISC,7.20,-28.05,80.43
ELECTRICAL,5.97,-11.75,52.68
PAINTS,5.84,-12.31,28.99
INSURANCE,4.72,-1.97,36.87
PHARMA,4.04,-3.06,35.50
AUTO,2.84,-31.07,76.51


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3136828.0,21
AR,1323265.0,10
XR,1270401.0,13
X40,971041.0,14
X40N,871121.0,9
OX40N,722996.0,10
XY25,353090.0,6
SR,280429.0,2
MH,76666.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3608944.0,25
M-SC,1433890.0,15
X-MC,1418975.0,16
X-LC,806240.0,11
X-SC,780652.0,8
H-MC,400773.0,3
L-SC,242066.0,3
M-LC,105585.0,1
H-LC,77680.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1248368.0      6
           AR         907767.0      5
M-SC       XY24       812259.0      6
H-SC       XR         773269.0      7
X-MC       X40        485993.0      7
           XY24       406633.0      3
           X40N       343659.0      4
H-SC       OX40N      322445.0      4
M-SC       OX40N      319669.0      5
X-SC       X40N       309051.0      3
X-LC       X40        306973.0      5
X-SC       XY24       293526.0      3
H-SC       SR         280429.0      2
X-LC       X40N       218411.0      2
H-MC       AR         212436.0      2
           XY24       188337.0      1
X-LC       XY24       187705.0      2
X-MC       XY25       182690.0      2
X-SC       X40        178075.0      2
M-SC       XR         176580.0      2
L-SC       XR         161184.0      2
M-SC       AR         125382.0      2
M-LC       XR         105585.0      1
X-LC       XY25        93151.0      2
L-SC       OX40N       80882.0      1
H-LC       AR          77680.0      1
H-SC       MH          76666.0      1
L-MC       XR          53783.0      1
M-MC       XY25        48065.0      1
L-LC       XY25        29184.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
